# Análisis de datos financieros

En finanzas, la creación de un portafolio de inversiones que se adecúe a las necesidades y perfil de una persona es de gran importancia, no sólo normativa si no para que duerma un poco más tranquila por las noches.

Para eso, es necesario revisar muchas variables, desde precios de activos, hasta cotizaciones de monedas o variables macroeconómicas. En este primer práctico vamos a entender un poco más sobre el dolar y sus diferentes cotizaciones, cómo podemos hacer un proxy de la misma para invertir en la bolsa y qué oportunidades de inversión podemos encontrar en base a eso.

Empecemos con la obtención de datos, a partir de la web de RAVA (hay un backup en el repo).

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import optimize
%matplotlib inline
import sys
from io import StringIO
# SOLO STOCKS QUE COMENZARON EN 2016
bonds = ['AO20', 'AO20D', 'AA37', 'AA37D', 'AY24', 'AY24D',
          'DICA', 'DICAD', 'AA25', 'AA25D', 'CO26', 'CO26D',
          'A2E2', 'A2E2D']

In [3]:
for bond in bonds:
    url = 'http://www.ravabursatil.com/v2/empresas/precioshistoricos.php?e={}&csv=1'.format(bond)
    req = requests.get(url, allow_redirects=True)
    df = pd.read_csv(StringIO(req.content.decode("utf-8")))
    name = '{}.csv'.format(bond)
    df.to_csv(name)

# Cómo seguimos?

Un bono es una especie de préstamo que pide un ente (nación, provincia, ciudad, compañía) con el objetivo de mejorar algo en su infraestructura o funcionamiento, con promesa de pagos de intereses en el camino a su vencimiento.

Para más información, se puede visitar [Este link (con chivo)](https://www.invera.com.ar) o, para más información aún, se puede usar [Investopedia](https://www.investopedia.com) para cualquier duda con respecto al mundo de las inversiones.

Las cotizaciones de los mismos están en dólares y pesos, para algunos en particular. Luego, pasaremos a revisar cotizaciones históricas del dólar.

In [4]:
df = pd.read_csv('dolarbna.csv')
df.head()

,Unnamed: 0,created,updated,currency,exchange_rate,exchange_date,exchange_rate_buy
0,0,2019-04-01 16:30:24,2019-04-01 19:30:24,USD_BN,43.7,2019-04-01 19:30:25,41.7
1,1,2019-04-01 16:00:26,2019-04-01 19:00:26,USD_BN,43.7,2019-04-01 19:00:27,41.7
2,2,2019-04-01 15:30:25,2019-04-01 18:30:25,USD_BN,43.7,2019-04-01 18:30:25,41.7
3,3,2019-04-01 15:00:25,2019-04-01 18:00:25,USD_BN,43.7,2019-04-01 18:00:27,41.7
4,4,2019-04-01 14:30:24,2019-04-01 17:30:24,USD_BN,43.7,2019-04-01 17:30:26,41.7


Como primera consigna, la cotización del dólar del BNA abre a las 10 AM y cierra a las 15 PM, podemos armar un dataset que, para cada día, tenga la cotización de cierre del precio del dólar. La columna de venta (la más vista por la gente) es la de 'exchange_rate'.

Vamos a cargar ahora las cotizaciones de los bonos y explorar un poquito... Revisemos el dataset del Bonar 2024 (AY24).

In [5]:
ay24 = pd.read_csv('AY24.csv')
ay24.head()

,Unnamed: 0,fecha,apertura,maximo,minimo,cierre,volumen,openint
0,0,2014-05-13,861.00000,880.00,861.0,880.00000,2756877,0
1,1,2014-05-14,883.00000,915.00,882.0,907.00000,23301196,0
2,2,2014-05-15,910.04999,927.45,899.5,922.00000,22532616,0
3,3,2014-05-16,925.00000,935.00,923.0,934.40002,32249204,0
4,4,2014-05-19,937.00000,974.00,936.0,970.00000,18602860,0


Empecemos a limpiar un poco los datasets.

- Tenemos algunas columnas que no sirven para nada, como 'Unnamed'.
- Es realmente inservible 'openint'? Sin saber qué significa, podemos decidir si nos va a servir para algo?
- Al igual que con el dólar, necesitamos un dataset que, para cada día tenga el último precio de negociación, indicado por la columna 'cierre'.
- Podemos plotear este dataset que creamos?

Es posible sacar alguna estadística útil para este dataset? Desde el ploteo no lo parece...
Lo importante en finanzas es ver cuánto varía el precio de un activo desde un cierre hasta el próximo, para eso existe el muy útil método pct_change().

Parece que va a ser más sencillo revisar algo en este espacio:

- Veamos estadísticas básicas (media, desviación estándar, moda).
- Una afirmación que se suele hacer con los retornos diarios de bonos y acciones, es que se asemejan bastante a una distribución normal. Es así en este caso?


Bueno, ahora que nos acomodamos un poco con los datos, vamos a leer el dataset de precios en dólares del AY24, cuyo ticker (abreviación) es AY24D y apliquemos lo mismo que hicimos antes.

Vamos a inventarnos una cotización del dólar "nuestra", dividamos el 